In [1]:
!pip install huggingface_hub

In [1]:
from huggingface_hub import interpreter_login
interpreter_login()  # This will prompt for your HF token


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [2]:
from huggingface_hub import HfApi, create_repo
import os
import shutil

def push_to_hub(checkpoint_path, repo_name, organization=None):
    """
    Push LDM checkpoints (including nested folders) to Hugging Face Hub
    """
    api = HfApi()
    full_repo_name = f"{organization}/{repo_name}" if organization else repo_name
    create_repo(full_repo_name, repo_type="model", exist_ok=True)

    temp_dir = "temp_upload"
    os.makedirs(temp_dir, exist_ok=True)

    try:
        for epoch_folder in os.listdir(checkpoint_path):
            epoch_path = os.path.join(checkpoint_path, epoch_folder)
            if os.path.isdir(epoch_path):
                epoch_temp_path = os.path.join(temp_dir, epoch_folder)
                os.makedirs(epoch_temp_path, exist_ok=True)

                for item in os.listdir(epoch_path):
                    src = os.path.join(epoch_path, item)
                    dst = os.path.join(epoch_temp_path, item)
                    if os.path.isfile(src):
                        shutil.copy2(src, dst)
                    elif os.path.isdir(src):
                        shutil.copytree(src, dst, dirs_exist_ok=True)

        api.upload_folder(
            folder_path=temp_dir,
            repo_id=full_repo_name,
            repo_type="model"
        )

        print(f"✅ Successfully pushed to: https://huggingface.co/{full_repo_name}")

    finally:
        shutil.rmtree(temp_dir)

# Example usage
push_to_hub("./ldm_checkpoints", "latent-diffusion-model-128x128", organization="vish26")


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✅ Successfully pushed to: https://huggingface.co/vish26/latent-diffusion-model-128x128
